In [1]:
import xml.etree.ElementTree as ET
import re
import requests
import keyring
import os
import map_functions_API
from testingRecord_API import check_mandatory_fields
# from testingRecord_API import write_log

In [2]:
# clear folder
filelist = [file for file in os.listdir("testfolder") if file.endswith(".txt")]
for f in filelist:
    os.remove(os.path.join("testfolder", f))

In [3]:
def loader(inputList):
    key = keyring.get_password("alma_api", "alx_prod").rstrip()
    base = "https://api-eu.hosted.exlibrisgroup.com"
    for acNr in inputList:
        url = f"{base}/almaws/v1/bibs?other_system_id=(AT-OBV){acNr}&apikey={key}"
        response = requests.get(url)
        root = ET.fromstring(response.content)
        # save response for checking
        with open(f"response_files/response_{acNr}.xml", "w", encoding="UTF-8") as f:
            f.write(response.text)
    return root

root = loader(["AC16712579","AC16712576"])
root.tag

'bibs'

In [5]:
# record.find(".//controlfield[@tag='008']").text

In [4]:
collection = root.find("bib")

In [21]:
with open("inputTXT.txt", "r") as i:
    textfile = i.read()
    for line in i.readlines()
        

    number = re.search("AC\d{8}", textfile)
    # lines = i.readlines()
print(textfile)

AttributeError: 'list' object has no attribute 'stip'

In [12]:
for number in lines:
    # root = loader(number)
    print(number)

AC16712579

AC16712576


In [9]:
def create_DCxml(record):
    '''
    Creates output root element 'resource'
    Goes through a Dictionary of datafield tags and starts the corresponding function
    '''
    # create output root
    output = ET.Element("resource")
    output.attrib = {
        "xmlns:xsi":"http://www.w3.org/2001/XMLSchema-instance",
        "xsi:schemaLocation": "http://datacite.org/schema/kernel-4 https://schema.datacite.org/meta/kernel-4.4/metadata.xsd",
        "xmlns":"http://datacite.org/schema/kernel-4"
        }
    # Dictionary to map functions to Alma Datafield
    tagsDict = {
        "008":map_functions_API.create_date,
        "024":map_functions_API.create_identifier,
        "041":map_functions_API.create_language,
        "100":map_functions_API.create_creator,
        "245":map_functions_API.create_title,
        "264":map_functions_API.create_publisher,
        "300":map_functions_API.create_size,
        "347":map_functions_API.create_formats,
        "520":map_functions_API.create_descriptions,
        "536":map_functions_API.create_fundingReference,
        "540":map_functions_API.create_rights,
        "700":map_functions_API.create_creator,
        "970":map_functions_API.create_resourceType
        }
    # if datafield is in Alma xml - run function
    for key in tagsDict:
        if record.find(".//controlfield[@tag='{}']".format(key)) != None:
            tagsDict[key](output, record)
        elif record.find(".//datafield[@tag='{}']".format(key)) != None:
            tagsDict[key](output, record)
    return output

def main():
    '''
    Loads the input xml file (Alma export), checks if mandatory fields are present and starts for each record in collection the create_DCxml function.
    '''
    # Load Alma xml
    # tree = ET.parse("inputFile.xml")
    with open("inputTXT.txt") as i:
        inputList = i.readlines()
    for number in inputList:
        root = loader(number)
        # collection = tree.getroot()
        #clear log file
        open("output/log.txt", "w").close()
        # go through all records in collection    
        counter = 0
        for record in root:
            check_mandatory_fields(record)
            output = create_DCxml(record)
            # create tree
            outputTree = ET.ElementTree(output)
            # write output
            acNr = record.find(".//controlfield[@tag='009']").text
            outputTree.write("output/output_{}.xml".format(acNr))
            print("created output_{}.xml".format(acNr))
            counter+=1
            # print(counter)
        
        print("completed {} files".format(counter))
        # count content in log file
        errorCount = 0
        with open("output/log.txt") as log:
            errorCount = len(log.readlines())
        print("{} caveats - see log file".format(errorCount))

In [10]:
main()

KeyboardInterrupt: 

In [ ]:
for record in root:
    # check_mandatory_fields(record)
    output = create_DCxml(record)
    # create tree
    outputTree = ET.ElementTree(output)
    # write output
    acNr = record.find(".//controlfield[@tag='009']").text
    outputTree.write("output_{}.xml".format(acNr))
    print("created output_{}.xml".format(acNr))
    # counter+=1
    # print(counter)

created output_AC16712576.xml
